# RSA NetWitness Respond API Notebook

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# version 1.0
# updated 2019/03/19

#RSA LInk API Guide
#https://community.rsa.com/docs/DOC-94420

import requests
from requests.auth import HTTPBasicAuth

import json
from datetime import datetime, timedelta
# disable insecure warnings
requests.packages.urllib3.disable_warnings()

#to read in the json as dataframe
import pandas as pd
import matplotlib.pyplot as plt

#allow plot inline charts
%matplotlib inline

path_auth = '/rest/api/auth/userpass'
path_incidents = '/rest/api/incidents'
path_alerts = '/alerts'

#------------ update these ----------------
# uri for the repond server and port (standard https port for Respond API)
url = 'https://<IP>:<optional port>'

#password and username to access the Respond API
user = '<user>'
password = '<password>'

In [ ]:
#$ curl 'https://api.netwitness.local/rest/api/auth/userpass' -i -X POST \
#  -H 'Accept: application/json;charset=UTF-8' \
#  -H 'Content-Type: application/x-www-form-urlencoded; charset=ISO-8859-1' \
#  -d 'username=ian&password=changeMe

In [ ]:
# POST /rest/api/auth/userpass

# you need an auth token to connect

payload = {'username': user,'password': password}
headers = {'NetWitness-Version': '1.0','content-type': 'x-www-form-urlencoded; charset=ISO-8859-1', 'Accept': 'application/json;charset=UTF-8'}

s = requests.Session()

#response = requests.post(url+path_auth, auth=HTTPBasicAuth('orchestrator', 'Password!1'), headers=headers, verify=False)
response = s.post(url+path_auth, verify=False, params=payload, headers=headers)
#print (response)

#print(response.status_code) 
#print(response.text)
#print(response.json())
access_result = response.json().get('accessToken')
refresh_result = response.json().get('refreshToken')
print(access_result)

In [ ]:
#GET /rest/api/incidents?since=2018-01-01T14:00:00.000Z

#use that auth token to connect and query for the incidents created in the last x days

#how many days back to get incidents
num_of_days = 30
if num_of_days:
    since = datetime.now() - timedelta(days=int(num_of_days))
    #convert to ISO 8601 format and add Z suffix
    timestamp =  since.isoformat() + 'Z'
    since = timestamp
    until = None
            
headers = {'NetWitness-Version': '1.0', 'Accept': 'application/json;charset=UTF-8','NetWitness-Token':access_result }

response = requests.get(url+path_incidents+'?since='+since, verify=False , headers=headers)        # To execute get request 
print(response.status_code)     # To print http response code  
#print (response.text)



In [ ]:
#get the response from the connection into json format

item = response.json().get('items')
#iterate over the list to print the items
for incident in item:
    #print(incident['id']+','+incident['title'] +','+incident['riskScore'])
    print(incident)

In [ ]:
#now make this as a datafrome for manipulation

df = pd.DataFrame.from_dict(item, orient='columns')
df.set_index('id', inplace=True)
df

In [ ]:
#count the values of incidents

#df.groupby('createdBy').count()
pd.value_counts(df['createdBy'].values, sort=True)

In [ ]:
#chart the values above
alertRiskScorePlot = pd.value_counts(df['averageAlertRiskScore'].values, sort=True).plot(kind='bar',title="Count averageAlertRiskScore of Incidents",legend=None,figsize=(9, 7))

In [ ]:
#unique items in the dataframe column selected
df['createdBy'].unique()

In [ ]:
#get the details for a specific Incident ID

#GET /rest/api/incidents/{id}

headers = {'NetWitness-Version': '1.0', 'Accept': 'application/json;charset=UTF-8','NetWitness-Token':access_result }

response_a = requests.get(url+path_incidents+'/INC-30', verify=False , headers=headers)        # To execute get request 
print(response_a.status_code)     # To print http response code  
print (response_a.text)

In [ ]:
#Get the alerts for a specific Incident

#GET /rest/api/incidents/{id}/alerts

headers = {'NetWitness-Version': '1.0', 'Accept': 'application/json;charset=UTF-8','NetWitness-Token':access_result }

response = requests.get(url+path_incidents+'/INC-30'+path_alerts, verify=False , headers=headers)        # To execute get request 
print(response.status_code)     # To print http response code  
print (response.text)